In [1]:
import os
import torch
import torchaudio
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
from scipy.io import wavfile
from scipy.fft import fft, fftfreq

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## FFT, 30-s

In [4]:
# class AudioDataset(Dataset):
#     def __init__(self, root_dir, sample_rate=16000, max_length=30*16000, exclude_files=None, set_type="train", train_ratio=0.2):
#         """
#         Args:
#             root_dir (str): Root directory with genre folders containing audio files.
#             sample_rate (int): Target sample rate for audio files.
#             max_length (int): Maximum length of each audio sequence in samples.
#             set_type (str): Specify "train" for training set and "test" for testing set.
#             train_ratio (float): The ratio of the data to use for training (e.g., 0.2 for 20% train, 80% test).
#         """
#         self.root_dir = root_dir
#         self.sample_rate = sample_rate
#         self.max_length = max_length
#         self.exclude_files = exclude_files if exclude_files else []
#         self.set_type = set_type
#         self.train_ratio = train_ratio
#         self.file_paths = []
#         self.labels = []
#         self.classes = sorted(os.listdir(root_dir))  # Get genre names as class labels

#         file_limit = 200
#         # Collect file paths and corresponding labels
#         for label_idx, genre in enumerate(self.classes):
#             genre_folder = os.path.join(root_dir, genre)
#             i = 0
#             for file_name in os.listdir(genre_folder):
#                 if i == file_limit:
#                     break
#                 if file_name.endswith(".wav") and file_name not in self.exclude_files:
#                     self.file_paths.append(os.path.join(genre_folder, file_name))
#                     self.labels.append(label_idx)
#                     i += 1

#         # Shuffle and split data
#         combined = list(zip(self.file_paths, self.labels))
#         random.shuffle(combined)
#         self.file_paths, self.labels = zip(*combined)

#         # Perform train-test split
#         split_index = int(len(self.file_paths) * self.train_ratio)
#         if self.set_type == "train":
#             # Use the first portion as the training set
#             self.file_paths = self.file_paths[:split_index]
#             self.labels = self.labels[:split_index]
#         else:
#             # Use the remaining portion as the testing set
#             self.file_paths = self.file_paths[split_index:]
#             self.labels = self.labels[split_index:]
    
#     def __len__(self):
#         return len(self.file_paths)
    
#     def load_audio(self, file_path):
#         """
#         Load audio file using torchaudio, and fall back to librosa if needed.
#         """
#         try:
#             waveform, sr = torchaudio.load(file_path)
#             if sr != self.sample_rate:
#                 waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=self.sample_rate)(waveform)
#         except RuntimeError as e:
#             print(f"torchaudio failed for {file_path} with error: {e}. Trying librosa as fallback.")
#             waveform, sr = librosa.load(file_path, sr=self.sample_rate)
#             waveform = torch.tensor(waveform).unsqueeze(0)  # Convert to PyTorch format, add channel dim
#         return waveform

#     def __getitem__(self, idx):
#         """
#         Get an audio sample and its label from the dataset.

#         Args:
#             idx (int): Index of the audio file to retrieve.

#         Returns:
#             tuple: A tuple containing the Fourier-transformed waveform and its corresponding label.
#         """
#         audio_path = self.file_paths[idx]
        
#         waveform = self.load_audio(audio_path)
#         if waveform is None:
#             print(f"Skipping {audio_path} due to load failure.")
#             return None

#         # Ensure mono audio
#         if waveform.size(0) > 1:
#             waveform = waveform.mean(dim=0, keepdim=True)

#         # Pad or truncate
#         num_samples = waveform.size(1)
#         if num_samples > self.max_length:
#             waveform = waveform[:, :self.max_length]
#         else:
#             padding = self.max_length - num_samples
#             waveform = torch.nn.functional.pad(waveform, (0, padding))

#         # Perform Fourier Transform and keep only the first half
#         waveform_np = waveform.numpy()  # Convert tensor to numpy array
#         fourier_transform = np.abs(fft(waveform_np[0]))[:self.max_length // 2]  # Only the first half

#         # Retrieve label
#         label = self.labels[idx]

#         return torch.tensor(fourier_transform, dtype=torch.float32).unsqueeze(0), label  # Add channel dimension

## 10-second Clips

In [6]:
# class AudioDataset(Dataset):
#     def __init__(self, root_dir, sample_rate=16000, segment_length=10*16000, exclude_files=None, set_type="train", train_ratio=0.2):
#         """
#         Args:
#             root_dir (str): Root directory with genre folders containing audio files.
#             sample_rate (int): Target sample rate for audio files.
#             segment_length (int): Length of each audio segment in samples (10 seconds).
#             set_type (str): Specify "train" for training set and "test" for testing set.
#             train_ratio (float): The ratio of the data to use for training (e.g., 0.2 for 20% train, 80% test).
#         """
#         self.root_dir = root_dir
#         self.sample_rate = sample_rate
#         self.segment_length = segment_length  # Length of each segment (10 seconds)
#         self.exclude_files = exclude_files if exclude_files else []
#         self.set_type = set_type
#         self.train_ratio = train_ratio
#         self.file_paths = []
#         self.labels = []
#         self.classes = sorted(os.listdir(root_dir))  # Get genre names as class labels

#         file_limit = 289
#         # Collect file paths and corresponding labels
#         for label_idx, genre in enumerate(self.classes):
#             genre_folder = os.path.join(root_dir, genre)
#             i = 0
#             for file_name in os.listdir(genre_folder):
#                 if i == file_limit:
#                     break
#                 if file_name.endswith(".wav") and file_name not in self.exclude_files:
#                     self.file_paths.append(os.path.join(genre_folder, file_name))
#                     self.labels.append(label_idx)
#                     i += 1

#         # Shuffle and split data
#         combined = list(zip(self.file_paths, self.labels))
#         random.shuffle(combined)
#         self.file_paths, self.labels = zip(*combined)

#         # Perform train-test split
#         split_index = int(len(self.file_paths) * self.train_ratio)
#         if self.set_type == "train":
#             # Use the first portion as the training set
#             self.file_paths = self.file_paths[:split_index]
#             self.labels = self.labels[:split_index]
#         else:
#             # Use the remaining portion as the testing set
#             self.file_paths = self.file_paths[split_index:]
#             self.labels = self.labels[split_index:]
    
#     def __len__(self):
#         return len(self.file_paths) * 3  # Each file is split into 3 segments
    
#     def load_audio(self, file_path):
#         """
#         Load audio file using torchaudio, and fall back to librosa if needed.
#         """
#         try:
#             waveform, sr = torchaudio.load(file_path)
#             if sr != self.sample_rate:
#                 waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=self.sample_rate)(waveform)
#         except RuntimeError as e:
#             print(f"torchaudio failed for {file_path} with error: {e}. Trying librosa as fallback.")
#             waveform, sr = librosa.load(file_path, sr=self.sample_rate)
#             waveform = torch.tensor(waveform).unsqueeze(0)  # Convert to PyTorch format, add channel dim
#         return waveform

#     def __getitem__(self, idx):
#         """
#         Get a 10-second audio segment and its label from the dataset.

#         Args:
#             idx (int): Index of the audio segment to retrieve.

#         Returns:
#             tuple: A tuple containing the Fourier-transformed waveform segment and its corresponding label.
#         """
#         # Determine file index and segment index
#         file_idx = idx // 3       # Index of the file in self.file_paths
#         segment_idx = idx % 3     # Segment within the file (0, 1, or 2 for 10-second clips)

#         # Load the waveform
#         audio_path = self.file_paths[file_idx]
#         waveform = self.load_audio(audio_path)
#         if waveform is None:
#             print(f"Skipping {audio_path} due to load failure.")
#             return None

#         # Ensure mono audio
#         if waveform.size(0) > 1:
#             waveform = waveform.mean(dim=0, keepdim=True)

#         # Calculate start and end for each 10-second segment
#         start = segment_idx * self.segment_length
#         end = start + self.segment_length
#         waveform_segment = waveform[:, start:end]

#         # If segment is shorter than segment_length, pad with zeros
#         num_samples = waveform_segment.size(1)
#         if num_samples < self.segment_length:
#             padding = self.segment_length - num_samples
#             waveform_segment = torch.nn.functional.pad(waveform_segment, (0, padding))

#         # Perform Fourier Transform on the segment and keep only the first half
#         waveform_np = waveform_segment.numpy()  # Convert tensor to numpy array
#         fourier_transform = np.abs(fft(waveform_np[0]))[:self.segment_length // 2]  # Only the first half

#         # Retrieve label
#         label = self.labels[file_idx]

#         return torch.tensor(fourier_transform, dtype=torch.float32).unsqueeze(0), label  # Add channel dimension

# 6-second clips

In [8]:
# class AudioDataset(Dataset):
#     def __init__(self, root_dir, sample_rate=16000, segment_length=6*16000, exclude_files=None, set_type="train", train_ratio=0.2):
#         """
#         Args:
#             root_dir (str): Root directory with genre folders containing audio files.
#             sample_rate (int): Target sample rate for audio files.
#             segment_length (int): Length of each audio segment in samples (6 seconds).
#             set_type (str): Specify "train" for training set and "test" for testing set.
#             train_ratio (float): The ratio of the data to use for training (e.g., 0.2 for 20% train, 80% test).
#         """
#         self.root_dir = root_dir
#         self.sample_rate = sample_rate
#         self.segment_length = segment_length  # Length of each segment (10 seconds)
#         self.exclude_files = exclude_files if exclude_files else []
#         self.set_type = set_type
#         self.train_ratio = train_ratio
#         self.file_paths = []
#         self.labels = []
#         self.classes = sorted(os.listdir(root_dir))  # Get genre names as class labels

#         file_limit = 289
#         # Collect file paths and corresponding labels
#         for label_idx, genre in enumerate(self.classes):
#             genre_folder = os.path.join(root_dir, genre)
#             i = 0
#             for file_name in os.listdir(genre_folder):
#                 if i == file_limit:
#                     break
#                 if file_name.endswith(".wav") and file_name not in self.exclude_files:
#                     self.file_paths.append(os.path.join(genre_folder, file_name))
#                     self.labels.append(label_idx)
#                     i += 1

#         # Shuffle and split data
#         combined = list(zip(self.file_paths, self.labels))
#         random.shuffle(combined)
#         self.file_paths, self.labels = zip(*combined)

#         # Perform train-test split
#         split_index = int(len(self.file_paths) * self.train_ratio)
#         if self.set_type == "train":
#             # Use the first portion as the training set
#             self.file_paths = self.file_paths[:split_index]
#             self.labels = self.labels[:split_index]
#         else:
#             # Use the remaining portion as the testing set
#             self.file_paths = self.file_paths[split_index:]
#             self.labels = self.labels[split_index:]
    
#     def __len__(self):
#         return len(self.file_paths) * 5  # Each file is split into 5 segments
    
#     def load_audio(self, file_path):
#         """
#         Load audio file using torchaudio, and fall back to librosa if needed.
#         """
#         try:
#             waveform, sr = torchaudio.load(file_path)
#             if sr != self.sample_rate:
#                 waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=self.sample_rate)(waveform)
#         except RuntimeError as e:
#             print(f"torchaudio failed for {file_path} with error: {e}. Trying librosa as fallback.")
#             waveform, sr = librosa.load(file_path, sr=self.sample_rate)
#             waveform = torch.tensor(waveform).unsqueeze(0)  # Convert to PyTorch format, add channel dim
#         return waveform

#     def __getitem__(self, idx):
#         """
#         Get a 5-second audio segment and its label from the dataset.

#         Args:
#             idx (int): Index of the audio segment to retrieve.

#         Returns:
#             tuple: A tuple containing the Fourier-transformed waveform segment and its corresponding label.
#         """
#         # Determine file index and segment index
#         file_idx = idx // 5       # Index of the file in self.file_paths
#         segment_idx = idx % 5     # Segment within the file (for 6-second clips)

#         # Load the waveform
#         audio_path = self.file_paths[file_idx]
#         waveform = self.load_audio(audio_path)
#         if waveform is None:
#             print(f"Skipping {audio_path} due to load failure.")
#             return None

#         # Ensure mono audio
#         if waveform.size(0) > 1:
#             waveform = waveform.mean(dim=0, keepdim=True)

#         # Calculate start and end for each 5-second segment
#         start = segment_idx * self.segment_length
#         end = start + self.segment_length
#         waveform_segment = waveform[:, start:end]

#         # If segment is shorter than segment_length, pad with zeros
#         num_samples = waveform_segment.size(1)
#         if num_samples < self.segment_length:
#             padding = self.segment_length - num_samples
#             waveform_segment = torch.nn.functional.pad(waveform_segment, (0, padding))

#         # Perform Fourier Transform on the segment and keep only the first half
#         waveform_np = waveform_segment.numpy()  # Convert tensor to numpy array
#         fourier_transform = np.abs(fft(waveform_np[0]))[:self.segment_length // 2]  # Only the first half

#         # Retrieve label
#         label = self.labels[file_idx]

#         return torch.tensor(fourier_transform, dtype=torch.float32).unsqueeze(0), label  # Add channel dimension

## 5-second clips

In [10]:
class AudioDataset(Dataset):
    def __init__(self, root_dir, sample_rate=16000, segment_length=5*16000, exclude_files=None, set_type="train", train_ratio=0.2):
        """
        Args:
            root_dir (str): Root directory with genre folders containing audio files.
            sample_rate (int): Target sample rate for audio files.
            segment_length (int): Length of each audio segment in samples (5 seconds).
            set_type (str): Specify "train" for training set and "test" for testing set.
            train_ratio (float): The ratio of the data to use for training (e.g., 0.2 for 20% train, 80% test).
        """
        self.root_dir = root_dir
        self.sample_rate = sample_rate
        self.segment_length = segment_length  # Length of each segment (5 seconds)
        self.exclude_files = exclude_files if exclude_files else []
        self.set_type = set_type
        self.train_ratio = train_ratio
        self.file_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dir))  # Get genre names as class labels

        file_limit = 289
        # Collect file paths and corresponding labels
        for label_idx, genre in enumerate(self.classes):
            genre_folder = os.path.join(root_dir, genre)
            i = 0
            for file_name in os.listdir(genre_folder):
                if i == file_limit:
                    break
                if file_name.endswith(".wav") and file_name not in self.exclude_files:
                    self.file_paths.append(os.path.join(genre_folder, file_name))
                    self.labels.append(label_idx)
                    i += 1

        # Shuffle and split data
        combined = list(zip(self.file_paths, self.labels))
        random.shuffle(combined)
        self.file_paths, self.labels = zip(*combined)

        # Perform train-test split
        split_index = int(len(self.file_paths) * self.train_ratio)
        if self.set_type == "train":
            # Use the first portion as the training set
            self.file_paths = self.file_paths[:split_index]
            self.labels = self.labels[:split_index]
        else:
            # Use the remaining portion as the testing set
            self.file_paths = self.file_paths[split_index:]
            self.labels = self.labels[split_index:]
    
    def __len__(self):
        return len(self.file_paths) * 6  # Each file is split into 6 segments
    
    def load_audio(self, file_path):
        """
        Load audio file using torchaudio
        """
        waveform, sr = torchaudio.load(file_path)
        if sr != self.sample_rate:
            waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=self.sample_rate)(waveform)
        return waveform

    def __getitem__(self, idx):
        """
        Get a 5-second audio segment and its label from the dataset.

        Args:
            idx (int): Index of the audio segment to retrieve.

        Returns:
            tuple: A tuple containing the Fourier-transformed waveform segment and its corresponding label.
        """
        # Determine file index and segment index
        file_idx = idx // 6       # Index of the file in self.file_paths
        segment_idx = idx % 6     # Segment within the file (for 5-second clips)

        # Load the waveform
        audio_path = self.file_paths[file_idx]
        waveform = self.load_audio(audio_path)
        if waveform is None:
            print(f"Skipping {audio_path} due to load failure.")
            return None

        # Ensure mono audio
        if waveform.size(0) > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # Calculate start and end for each 5-second segment
        start = segment_idx * self.segment_length
        end = start + self.segment_length
        waveform_segment = waveform[:, start:end]

        # If segment is shorter than segment_length, pad with zeros
        num_samples = waveform_segment.size(1)
        if num_samples < self.segment_length:
            padding = self.segment_length - num_samples
            waveform_segment = torch.nn.functional.pad(waveform_segment, (0, padding))

        # Perform Fourier Transform on the segment and keep only the first half
        waveform_np = waveform_segment.numpy()  # Convert tensor to numpy array
        fourier_transform = np.abs(fft(waveform_np[0]))[:self.segment_length // 2]  # Only the first half

        # Retrieve label
        label = self.labels[file_idx]

        return torch.tensor(fourier_transform, dtype=torch.float32).unsqueeze(0), label  # Add channel dimension

## Model 1

In [12]:
# # # Define simple and lame 1D CNN model for audio classification
# # # add a second fully connected layer

# class AudioCNN(nn.Module):
#     def __init__(self, input_length=30*16000):
#         super(AudioCNN, self).__init__()
        
#         # Define convolutional layers
#         self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
#         self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
#         # Define pooling layer
#         self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

#         # Calculate the flattened size after the last pooling layer
#         # Pass a dummy input through the conv and pool layers to find the shape
#         with torch.no_grad():
#             dummy_input = torch.zeros(1, 1, input_length)  # Batch size 1, 1 channel, input length
#             out = self.pool(F.relu(self.conv1(dummy_input)))
#             out = self.pool(F.relu(self.conv2(out)))
#             out = self.pool(F.relu(self.conv3(out)))
#             self.flattened_size = out.shape[1] * out.shape[2]  # Channels * Width after conv/pooling layers

#         # Define fully connected layers using the computed flattened size
#         self.fc1 = nn.Linear(self.flattened_size, 128)  # Match flattened size here
#         self.fc2 = nn.Linear(128, 32)  # Match flattened size here
#         self.fc3 = nn.Linear(32, 10)  # Example output layer, e.g., for 10 classes

#     def forward(self, x):
#         # Pass through convolutional and pooling layers
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.pool(F.relu(self.conv3(x)))
        
#         # Flatten the tensor to match the input of the fully connected layer
#         x = x.view(x.size(0), -1)  # Flatten all dimensions except batch
#         x = F.relu(self.fc1(x))  # Pass through first fully connected layer
#         x = F.relu(self.fc2(x))  # Pass through second fully connected layer
#         x = self.fc3(x)          # Output layer
        
#         return x

# # Accuracy of the model on the test set (1s clips, 1 fc layer): 31.54%
# # Accuracy of the model on the test set (30s clips, 1 fc layer): 33.26% (3.4507, 2.0044, 1.3442, 0.6947, 0.2494)
# # Accuracy of the model on the test set (100 30s clips, 2 fc layer): 16.12% (15.5413, 4.4464, 2.5460, 2.0580, 1.7405)
# # Accuracy of the model on the test set (200 30s clips, 2 fc layer): 30.06% (5.9436, 2.3099, 2.2481, 1.6181, 0.8132)

## Model 2

In [14]:
# # Define a 1D CNN model for audio classification that imitates the 2D
# # give up on this one because of memory issues
# class AudioCNN(nn.Module):
#     def __init__(self, input_length=30*16000):
#         super(AudioCNN, self).__init__()
        
#         # Define convolutional layers
#         self.conv1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding='same')
#         self.conv2 = nn.Conv1d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding='same')

#         # Define pooling layer
#         self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

#         # Define convolutional layers
#         self.conv3 = nn.Conv1d(in_channels=8, out_channels=12, kernel_size=3, stride=1, padding='same')
#         self.conv4 = nn.Conv1d(in_channels=12, out_channels=16, kernel_size=3, stride=1, padding='same')

#         # Define subsampling layer
#         self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

#         # # Define convolutional layers
#         # self.conv5 = nn.Conv1d(in_channels=16, out_channels=20, kernel_size=3, stride=1, padding='same')
#         # self.conv6 = nn.Conv1d(in_channels=20, out_channels=24, kernel_size=3, stride=1, padding='same')

#         # # Define subsampling layer
#         # self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)

#         # Calculate the flattened size after the last pooling layer
#         # Pass a dummy input through the conv and pool layers to find the shape
#         with torch.no_grad():
#             dummy_input = torch.zeros(1, 1, input_length)  # Batch size 1, 1 channel, input length
#             out = F.relu(self.conv1(dummy_input))
#             out = F.relu(self.conv2(out))
#             out = self.pool1(out)  # Apply pool1 after conv2
#             out = F.relu(self.conv3(out))
#             out = F.relu(self.conv4(out))
#             out = self.pool2(out)  # Apply pool2 after conv4
#             # out = F.relu(self.conv5(out))
#             # out = F.relu(self.conv6(out))
#             # out = self.pool3(out)  # Apply pool3 after conv6
#             self.flattened_size = out.shape[1] * out.shape[2]  # Channels * Width after conv/pooling layers

        
#         # Define fully connected layers using the computed flattened size
#         self.fc1 = nn.Linear(in_features=self.flattened_size, out_features=5000)
#         # self.fc2 = nn.Linear(in_features=5000, out_features=1000)
#         # self.fc3 = nn.Linear(in_features=1000, out_features=100)
#         self.fc4 = nn.Linear(in_features=5000, out_features=10)


#     def forward(self, x):
#         # Pass through convolutional and pooling layers
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = self.pool1(x)
#         x = F.relu(self.conv3(x))
#         x = F.relu(self.conv4(x))
#         x = self.pool2(x)
#         # x = F.relu(self.conv5(x))
#         # x = F.relu(self.conv6(x))
#         # x = self.pool3(x)

#         # Flatten the tensor to match the input of the fully connected layer
#         x = x.view(x.size(0), -1)  # Flatten all dimensions except batch
#         x = F.relu(self.fc1(x))  # Pass through first fully connected layer
#         # x = F.relu(self.fc2(x))  # Pass through first fully connected layer
#         # x = F.relu(self.fc3(x))  # Pass through first fully connected layer
#         x = self.fc4(x)          # Output layer
#         return x

# # Accuracy of the model on the test set (30s clips): 10.07%

## Model 3

In [16]:
# # Define a 1D CNN model for Fourier-transformed audio data with multiple input channels
# class AudioCNN(nn.Module):
#     def __init__(self, input_length=30*16000 // 2):
#         super(AudioCNN, self).__init__()
        
#         self.conv1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding='same')
#         self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

#         self.conv2 = nn.Conv1d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding='same')
#         self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

#         # Calculate the flattened size after the last pooling layer
#         with torch.no_grad():
#             dummy_input = torch.zeros(1, 1, input_length)  # Batch size 1, 1 channel, input length
#             out = F.relu(self.conv1(dummy_input))
#             out = self.pool1(out)
#             out = F.relu(self.conv2(out))
#             out = self.pool2(out) 
#             self.flattened_size = out.shape[1] * out.shape[2]  # Channels * Width after conv/pooling layers

#         # Define fully connected layers using the computed flattened size
#         self.fc1 = nn.Linear(in_features=self.flattened_size, out_features=1000)
#         self.fc2 = nn.Linear(in_features=1000, out_features=10)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = self.pool1(x)
#         x = F.relu(self.conv2(x))
#         x = self.pool2(x)

#         x = x.view(x.size(0), -1)  # Flatten all dimensions except batch
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

# # Accuracy of the model on the test set (30 s, fft, batch32): 42.94% (626.5556, 51.8651, 3.5706)
# # Accuracy of the model on the test set (30, fft, batch32): 44.25% (60.5790, 4.5441, 2.2963, 0.5291, 0.0785)

## Model 4

In [18]:
# Define another 1D CNN model for Fourier-transformed audio data
class AudioCNN(nn.Module):
    def __init__(self, input_length=5*16000 // 2):
        super(AudioCNN, self).__init__()
        
        # First convolutional layer and pooling
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding='same')
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        # Second convolutional layer and pooling
        self.conv2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding='same')
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        # Third convolutional layer and pooling
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding='same')
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Fourth convolutional layer and pooling
        self.conv4 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding='same')
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Fifth convolutional layer and pooling
        self.conv5 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding='same')
        self.pool5 = nn.MaxPool1d(kernel_size=2, stride=2)

        # Calculate the flattened size after the last pooling layer
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, input_length)  # Batch size 1, 1 channel, input length
            out = F.relu(self.conv1(dummy_input))
            out = self.pool1(out)
            out = F.relu(self.conv2(out))
            out = self.pool2(out)
            out = F.relu(self.conv3(out))
            out = self.pool3(out)
            out = F.relu(self.conv4(out))
            out = self.pool4(out)
            out = F.relu(self.conv5(out))
            out = self.pool5(out)
            self.flattened_size = out.shape[1] * out.shape[2]  # Channels * Width after conv/pooling layers

        # Define fully connected layers using the computed flattened size
        self.fc1 = nn.Linear(in_features=self.flattened_size, out_features=3200)
        self.fc2 = nn.Linear(in_features=3200, out_features=1600)
        self.fc3 = nn.Linear(in_features=1600, out_features=800)
        self.fc4 = nn.Linear(in_features=800, out_features=10)

    def forward(self, x):
        # Pass through convolutional and pooling layers
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = F.relu(self.conv4(x))
        x = self.pool4(x)

        # Flatten the tensor to match the input of the fully connected layer
        x = x.view(x.size(0), -1)  # Flatten all dimensions except batch
        x = F.relu(self.fc1(x))    # Pass through first fully connected layer
        x = F.relu(self.fc2(x))    # Pass through second fully connected layer
        x = F.relu(self.fc3(x))    # Pass through third fully connected layer
        x = self.fc4(x)            # Output layer
        return x

# Accuracy of the model on the test set (30s, 2cp+2fc): 49.94% (259.7404, 43.7874, 2.2371, 0.4296, 0.0983)
# Accuracy of the model on the test set (30s, 3cp+2fc): 46.44% (665.3874, 170.0507, 69.5730, 25.9738, 4.6241)
# Accuracy of the model on the test set (30s, 10epoch, 2cp+2fc): 51.38% (8451.4932, 969.9626, 234.5838, 81.7483, 4.4840, 6.5689, 0.9066, 2.7939, 0.7037, 1.3528)
# Accuracy of the model on the test set (10s, 10epoch, 2cp+2fc): 51.47% (28.3695, 0.4100, 0.5335, 0.2201, 0.1588, 0.2497, 3.6753, 0.0198, 0.0388, 4.8844)
# Accuracy of the model on the test set (10s, 10epoch, 3cp+2fc): 57.58% (2.2222, 1.1562, 0.2161, 0.1510, 0.0169, 0.0003, 0.0004, 0.0000, 0.0002, 0.0001)
# Accuracy of the model on the test set (10s, 10epoch, 3cp+3fc): 59.18% (2.4068, 0.8191, 0.4002, 0.4372, 0.1581, 0.0804, 0.0336, 0.0566, 0.1198, 0.0088)
# Accuracy of the model on the test set (6s, 10epoch, 3cp+3fc): 57.18% (1.3956, 0.6558, 0.0348, 0.0102, 0.0113, 0.0274, 0.0019, 0.0001, 0.0105, 0.0013)
# Accuracy of the model on the test set (6s, 10epoch, 4cp+3fc): 57.89% (1.1886, 1.2358, 0.1747, 0.5024, 0.0492, 0.1019, 0.0037, 0.0001, 0.0006, 0.000057)
# Accuracy of the model on the test set (6s, 10epoch, 4cp+4fc): 49.52% (1.6578, 1.4446, 0.8011, 0.1661, 0.2279, 0.0614, 0.0247, 0.0033, 0.0036, 0.0100)
# Accuracy of the model on the test set (6s, 10epoch, 5cp+3fc): 54.86% (1.2255, 1.7275, 0.3796, 0.1867, 0.0577, 0.0010, 0.0031, 0.0214, 0.0040, 0.0087)
# Accuracy of the model on the test set (5s, 10epoch, 5cp+3fc): 59.83% (1.1961, 0.8041, 0.6351, 0.0491, 0.1683, 0.0060, 0.0005, 0.0004, 0.0001, 0.0003)
# Accuracy of the model on the test set (5s, 10epoch, 5cp+4fc): 55.64% (1.4656, 0.9191, 0.9812, 0.0128, 0.0013, 0.0102, 0.0771, 0.0010, 0.0003, 0.0011)
# Accuracy of the model on the test set (5s, 10epoch, 5cp+4fc, bigger fc): 58.55% (1.8197, 1.1054, 0.4446, 0.0244, 0.0141, 0.0133, 0.0765, 0.0001, 0.0003, 0.0001)
# Accuracy of the model on the test set (5s, 10epoch, 4cp+4fc, bigger cp): 56.18% (1.2513, 0.8952, 0.2364, 0.1094, 0.1831, 0.0042, 0.0216, 0.3237, 0.0035, 0.0020)
# Accuracy of the model on the test set (5s, 7epoch, 5cp+4fc): 55.83% (1.4570, 1.4312, 0.1253, 0.0622, 0.4947, 0.2065, 0.0038)
# Accuracy of the model on the test set (5s, 10epoch, 5cp+4fc, bigger fc) : 56.87% (1.5426, 0.9646, 0.2376, 0.2390, 0.0408, 0.0038, 0.0624, 0.0001, 0.0002, 0.0001)
# Accuracy of the model on the test set (5s, 10epoch, 5cp+4fc, 20filesfromeachgenre): 47.40%
# Accuracy of the model on the test set: 57.54%

## Model 5

In [20]:
# # Define another 1D CNN model for Fourier-transformed audio data
# class AudioCNN(nn.Module):
#     def __init__(self, input_length=5*16000 // 2):
#         super(AudioCNN, self).__init__()
        
#         # First convolutional layer and pooling
#         self.conv1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding='same')
#         self.conv2 = nn.Conv1d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding='same')
#         self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

#         # Second convolutional layer and pooling
#         self.conv3 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding='same')
#         self.conv4 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding='same')
#         self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

#         # Third convolutional layer and pooling
#         self.conv5 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding='same')
#         self.conv6 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding='same')
#         self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
#         # Calculate the flattened size after the last pooling layer
#         with torch.no_grad():
#             dummy_input = torch.zeros(1, 1, input_length)  # Batch size 1, 1 channel, input length
            
#             out = F.relu(self.conv1(dummy_input))
#             out = F.relu(self.conv2(out))
#             out = self.pool1(out)
            
#             out = F.relu(self.conv3(out))
#             out = F.relu(self.conv4(out))
#             out = self.pool2(out)

#             out = F.relu(self.conv5(out))
#             out = F.relu(self.conv6(out))
#             out = self.pool3(out)
            
#             self.flattened_size = out.shape[1] * out.shape[2]  # Channels * Width after conv/pooling layers

#         # Define fully connected layers using the computed flattened size
#         self.fc1 = nn.Linear(in_features=self.flattened_size, out_features=2000)
#         self.fc2 = nn.Linear(in_features=2000, out_features=10)
#         # self.fc3 = nn.Linear(in_features=2400, out_features=800)
#         # self.fc4 = nn.Linear(in_features=800, out_features=10)

#     def forward(self, x):
#         # Pass through convolutional and pooling layers
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = self.pool1(x)

#         x = F.relu(self.conv3(x))
#         x = F.relu(self.conv4(x))
#         x = self.pool2(x)

#         x = F.relu(self.conv5(x))
#         x = F.relu(self.conv6(x))
#         x = self.pool3(x)
        
#         # Flatten the tensor to match the input of the fully connected layer
#         x = x.view(x.size(0), -1)  # Flatten all dimensions except batch
#         x = F.relu(self.fc1(x))    # Pass through first fully connected layer
#         # x = F.relu(self.fc2(x))    # Pass through second fully connected layer
#         # x = F.relu(self.fc3(x))    # Pass through third fully connected layer
#         x = self.fc2(x)            # Output layer
#         return x



In [21]:
# Set root directory for the data
root_dir = "genres_original"

# 1. Create training and testing datasets
train_dataset = AudioDataset(root_dir=root_dir, set_type="train", exclude_files=['jazz.00054.wav'])
test_dataset = AudioDataset(root_dir=root_dir, set_type="test", exclude_files=['jazz.00054.wav'])

# 2. Create a DataLoader only for the training dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Optionally, a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [22]:
# Instantiate model, define loss function and optimizer
model = AudioCNN()
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

AudioCNN(
  (conv1): Conv1d(1, 8, kernel_size=(3,), stride=(1,), padding=same)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(8, 16, kernel_size=(3,), stride=(1,), padding=same)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=same)
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=same)
  (pool4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=same)
  (pool5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=160000, out_features=3200, bias=True)
  (fc2): Linear(in_features=3200, out_features=1600, bias=True)
  (fc3): Linear(in_features=1600, out_features=800, bias=True)
  (fc4): Linea

In [23]:
num_epochs = 10

# 3. Training loop
for epoch in range(num_epochs):
    for batch in train_loader:  # Only using train_loader for training loop
        waveforms, labels = batch
        waveforms, labels = waveforms.to(device), labels.to(device)

        # Forward pass
        outputs = model(waveforms)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
    
print("Training complete.")

Epoch [1/10], Loss: 1.8042
Epoch [2/10], Loss: 0.4773
Epoch [3/10], Loss: 0.1926
Epoch [4/10], Loss: 0.0476
Epoch [5/10], Loss: 0.2593
Epoch [6/10], Loss: 0.0005
Epoch [7/10], Loss: 0.0021
Epoch [8/10], Loss: 0.0003
Epoch [9/10], Loss: 0.0051
Epoch [10/10], Loss: 0.0006
Training complete.


In [24]:
model_path = "MusicGenreClassifier1D_M4.pth"
torch.save(model.state_dict(), model_path)

In [25]:
# Evaluation loop to calculate accuracy
def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation
        for waveforms, labels in test_loader:
            waveforms, labels = waveforms.to(device), labels.to(device)

            # Forward pass
            outputs = model(waveforms)

            # Get the predicted class (index of max log-probability)
            _, predicted = torch.max(outputs.data, 1)

            # Update total and correct counts
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total  # Calculate accuracy
    return accuracy

# Assuming you have already created test_loader, and trained your model
accuracy = evaluate_model(model, test_loader, device)
print(f'Accuracy of the model on the test set: {accuracy:.2f}%')

Accuracy of the model on the test set: 57.54%
